In [50]:
# post conuhacks note: pretty sure this file is useless

import pretty_midi as pm

midi_pretty_format = pm.PrettyMIDI('2004/MIDI-Unprocessed_XP_09_R1_2004_05_ORIG_MID--AUDIO_09_R1_2004_08_Track08_wav.midi')
print(len(midi_pretty_format.instruments[0].notes))
print(midi_pretty_format.instruments[0].notes)


6699
[Note(start=1.066667, end=1.169792, pitch=39, velocity=83), Note(start=1.072917, end=1.238542, pitch=51, velocity=93), Note(start=1.073958, end=1.255208, pitch=63, velocity=93), Note(start=1.194792, end=2.071875, pitch=49, velocity=94), Note(start=1.196875, end=2.303125, pitch=37, velocity=89), Note(start=1.196875, end=2.336458, pitch=61, velocity=95), Note(start=2.782292, end=2.911458, pitch=51, velocity=97), Note(start=2.784375, end=2.926042, pitch=39, velocity=81), Note(start=2.783333, end=2.939583, pitch=63, velocity=95), Note(start=2.903125, end=3.688542, pitch=47, velocity=91), Note(start=2.905208, end=3.692708, pitch=35, velocity=87), Note(start=2.904167, end=4.362500, pitch=59, velocity=90), Note(start=4.489583, end=4.628125, pitch=39, velocity=88), Note(start=4.483333, end=4.653125, pitch=63, velocity=93), Note(start=4.482292, end=4.822917, pitch=51, velocity=94), Note(start=4.635417, end=5.490625, pitch=46, velocity=97), Note(start=4.643750, end=5.568750, pitch=34, veloc

In [21]:
import numpy as np
piano_midi = midi_pretty_format.instruments[0] # Get the piano channels
piano_roll = piano_midi.get_piano_roll(fs=100)
print(piano_roll.shape)

# Convert 2D array to 1D
flattened = piano_roll.flatten()

# Get unique values
unique_values = np.unique(flattened)

# Print the unique values
print("Unique values in the array are: ", unique_values)

(128, 76814)
Unique values in the array are:  [  0.   6.   7.   8.   9.  12.  14.  15.  16.  18.  19.  20.  21.  22.
  23.  24.  25.  26.  27.  28.  29.  30.  31.  32.  33.  34.  35.  36.
  37.  38.  39.  40.  41.  42.  43.  44.  45.  46.  47.  48.  49.  50.
  51.  52.  53.  54.  55.  56.  57.  58.  59.  60.  61.  62.  63.  64.
  65.  66.  67.  68.  69.  70.  71.  72.  73.  74.  75.  76.  77.  78.
  79.  80.  81.  82.  83.  84.  85.  86.  87.  88.  89.  90.  91.  92.
  93.  94.  95.  96.  97.  98.  99. 100. 101. 102. 103. 105.]


In [ ]:
dict_time_notes = {"0": piano_roll}

print(dict_time_notes)

list_of_dict_keys_time = []
    
for key in dict_time_notes:
    sample = dict_time_notes[key]
    times = np.unique(np.where(sample > 0)[1])
    index = np.where(sample > 0)
    dict_keys_time = {}
    
    for time in times:
        index_where = np.where(index[1] == time)
        notes = index[0][index_where]
        dict_keys_time[time] = notes
    list_of_dict_keys_time.append(dict_keys_time)

print(list_of_dict_keys_time)

In [13]:
note_tokenizer = NoteTokenizer()

for note in list_of_dict_keys_time:
    note_tokenizer.partial_fit(list(note.values()))

# note_tokenizer.add_new_note('e') # Add empty notes

unique_notes = note_tokenizer.unique_word
print(unique_notes)


607


In [12]:
class NoteTokenizer:
    
    def __init__(self):
      self.notes_to_index = {}
      self.index_to_notes = {}
      self.num_of_word = 0
      self.unique_word = 0
      self.notes_freq = {}
        
    def transform(self,list_array):
      """ Transform a list of note in string into index.
      
      Parameters
      ==========
      list_array : list
        list of note in string format
      
      Returns
      =======
      The transformed list in numpy array.
      
      """
      transformed_list = []
      for instance in list_array:
          transformed_list.append([self.notes_to_index[note] for note in instance])
      return np.array(transformed_list, dtype=np.int32)

    def partial_fit(self, notes):
        """ Partial fit on the dictionary of the tokenizer
        
        Parameters
        ==========
        notes : list of notes
        
        """
        for note in notes:
            note_str = ','.join(str(a) for a in note)
            if note_str in self.notes_freq:
                self.notes_freq[note_str] += 1
                self.num_of_word += 1
            else:
                self.notes_freq[note_str] = 1
                self.unique_word += 1
                self.num_of_word += 1
                self.notes_to_index[note_str], self.index_to_notes[self.unique_word] = self.unique_word, note_str
            
    def add_new_note(self, note):
        """ Add a new note into the dictionary

        Parameters
        ==========
        note : str
          a new note who is not in dictionary.  

        """
        assert note not in self.notes_to_index
        self.unique_word += 1
        self.notes_to_index[note], self.index_to_notes[self.unique_word] = self.unique_word, note